In [5]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = ",".join([
    '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0',
    'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'])

if 'spark' in globals():
    spark.stop()
else:
    print("spark is not up yet")

spark = (
    SparkSession
    .builder
    .appName("Read Kafka")
    .config('spark.sql.shuffle.partitions', 4)
    .config('sspark.default.parallelism', 4)
    .config('spark.jars.packages', ','.join([
        'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0',
        'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0']))
    .config('spark.jars', ','.join([
        '/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/spark-sql-kafka-0-10_2.12-3.5.0.jar',
        '/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/spark-streaming-kafka-0-10_2.12-3.5.0.jar',
        '/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/kafka-clients-3.5.0.jar',
        '/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/spark-token-provider-kafka-0-10_2.12-3.5.0.jar',
        '/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/commons-pool2-2.8.0.jar']))
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
spark


spark is not up yet


23/11/09 19:39:54 WARN Utils: Your hostname, mac.local resolves to a loopback address: 127.0.0.1; using 10.251.181.122 instead (on interface en0)
23/11/09 19:39:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/hims/.ivy2/cache
The jars for the packages stored in: /Users/hims/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-65e28887-0709-4012-bcf9-7984bd12a37b;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_

In [1]:
from pyspark.sql.functions import from_json, col

def read_from_topic(topic):
    print(f"reading data from the topic {topic}")
    df = (
        spark
        .readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "localhost:9092")
        .option("startingOffsets", "earliest")
        .option("subscribe", topic)
        .load()
    )
    print(f"is spark is reading the streams from kafka = {df.isStreaming}")
    df.printSchema()
    return df.withColumn("value", col("value").cast("int"))

In [ ]:
def read_reviews():
    stream_df = read_from_topic("reviews")
    schema = StructType([
        StructField("business_id", StringType(), True),
        StructField("cool", LongType(), True),
        StructField("date", StringType(), True),
        StructField("funny", LongType(), True),
        StructField("review_id", StringType(), True),
        StructField("stars", DoubleType(), True),
        StructField("text", StringType(), True),
        StructField("useful", LongType(), True),
        StructField("user_id", StringType(), True),
    ])
    df_result = stream_df.select(from_json(col("json_string"), schema).alias("data"))
    return df_result

In [2]:
def read_checkins():
    stream_df = read_from_topic("checkins")
    schema = StructType([
        StructField("business_id", StringType(), True),
        StructField("date", StringType(), True),
    ])
    df_result = stream_df.select(from_json(col("json_string"), schema).alias("data"))
    return df_result


In [ ]:
def read_tips():
    stream_df = read_from_topic("tips")
    schema = StructType([
        StructField("business_id", StringType(), True),
        StructField("compliment_count", LongType(), True),
        StructField("date", StringType(), True),
        StructField("text", StringType(), True),
        StructField("user_id", StringType(), True),
    ])
    df_result = stream_df.select(from_json(col("json_string"), schema).alias("data"))
    return df_result